<a href="https://colab.research.google.com/github/farhanzibran/Top-Export-Import/blob/main/Top_Export_%26_Import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from IPython.display import display

# Membaca file CSV / excel ke dalam DataFrame
df = pd.read_excel('/content/Data Dasar 2.xlsx')

# Mengatur Pandas agar menampilkan semua baris
pd.set_option('display.max_rows', None)

# Mengatur Pandas agar menampilkan semua kolom
pd.set_option('display.max_columns', None)

# Daftar kolom yang ingin dihitung total sum-nya
columns_to_sum = [
    'SIZED_20_FULL_DISC', 'SIZED_40_FULL_DISC', 'SIZED_45_FULL_DISC',
    'SIZED_20_EMPTY_DISC', 'SIZED_40_EMPTY_DISC', 'SIZED_45_EMPTY_DISC',
    'SIZED_20_FULL_LOAD', 'SIZED_40_FULL_LOAD', 'SIZED_45_FULL_LOAD',
    'SIZED_20_EMPTY_LOAD', 'SIZED_40_EMPTY_LOAD', 'SIZED_45_EMPTY_LOAD'
]

# Mengisi nilai NaN dengan 0 (agar pivot table memperlakukan NaN sebagai 0)
df.fillna(0, inplace=True)

# Membuat pivot table berdasarkan 'COUNTRY_NAME', dengan total sum untuk kolom tertentu
pivot_df = pd.pivot_table(
    df,  # DataFrame utama
    index=['COUNTRY NAME'],  # Grup berdasarkan kolom COUNTRY_NAME
    values=columns_to_sum,  # Hanya menghitung sum untuk kolom-kolom ini
    aggfunc='sum',  # Agregasi penjumlahan
    margins=True,  # Menambahkan total sum keseluruhan (opsional)
    margins_name='Total'  # Nama untuk total sum keseluruhan
)
# Menghapus baris yang kosong pada kolom 'COUNTRY_NAME'
df = df[df['COUNTRY NAME'].notna()]
# Menghapus baris dengan index yang bernilai NaN atau 0 setelah pivot table
pivot_df = pivot_df[pivot_df.index != 0]  # Jika nilai index '0' muncul
pivot_df = pivot_df.dropna(how='all')  # Menghapus baris yang seluruhnya kosong
# Menggunakan assignment langsung tanpa inplace=True
df['COUNTRY NAME'] = df['COUNTRY NAME'].fillna('Unknown Country')

#display(pivot_df)

# Menyimpan hasil pivot table ke file CSV dan Excel
#pivot_df.to_csv('pivot_bongkar_muat_specific_sum.csv')
#pivot_df.to_excel('pivot.xlsx')


# Menambahkan kolom 'import full teus' sebagai jumlah dari SIZED_20_FULL_DISC, SIZED_40_FULL_DISC, dan SIZED_45_FULL_DISC
pivot_df['import full teus'] = (
    pivot_df['SIZED_20_FULL_DISC'] * 1 +
    pivot_df['SIZED_40_FULL_DISC'] * 2 +
    pivot_df['SIZED_45_FULL_DISC'] * 2.25
).astype(int)

# Menambahkan kolom 'import empty teus' sebagai jumlah dari SIZED_20_EMPTY_DISC, SIZED_40_EMPTY_DISC, dan SIZED_45_EMPTY_DISC
pivot_df['import empty teus'] = (
    pivot_df['SIZED_20_EMPTY_DISC'] * 1 +
    pivot_df['SIZED_40_EMPTY_DISC'] * 2 +
    pivot_df['SIZED_45_EMPTY_DISC'] * 2.25
).astype(int)

# Menambahkan kolom 'total teus' sebagai jumlah dari 'import full teus' dan 'import empty teus'
pivot_df['total import teus'] = pivot_df['import full teus'] + pivot_df['import empty teus']



# Menambahkan kolom 'export full teus' sebagai jumlah dari SIZED_20_FULL_LOAD, SIZED_40_FULL_LOAD, dan SIZED_45_FULL_LOAD
pivot_df['export full teus'] = (
    pivot_df['SIZED_20_FULL_LOAD'] * 1 +
    pivot_df['SIZED_40_FULL_LOAD'] * 2 +
    pivot_df['SIZED_45_FULL_LOAD'] * 2.25
).astype(int)

# Menambahkan kolom 'export empty teus' sebagai jumlah dari SIZED_20_EMPTY_LOAD, SIZED_40_EMPTY_LOAD, dan SIZED_45_EMPTY_LOAD
pivot_df['export empty teus'] = (
    pivot_df['SIZED_20_EMPTY_LOAD'] +
    pivot_df['SIZED_40_EMPTY_LOAD'] *2 +
    pivot_df['SIZED_45_EMPTY_LOAD'] *2.25
).astype(int)

# Menambahkan kolom 'total teus' (ekspor) sebagai jumlah dari 'export full teus' dan 'export empty teus'
pivot_df['total export teus'] = pivot_df['export full teus'] + pivot_df['export empty teus']

# Menambahkan kolom 'total EX&IMP teus' sebagai jumlah dari 'total teus' (impor) dan 'total teus' (ekspor)
pivot_df['total EX&IMP teus'] = pivot_df['total import teus'] + pivot_df['total export teus']

# Menghapus baris "Total" dari pivot table
pivot_df = pivot_df.drop(index='Total')

# Menampilkan hasil pivot table dengan kolom baru
#display(pivot_df)

# Menampilkan TOP 13 origin/import berdasarkan 'total import teus'
top_13_import = pivot_df.nlargest(13, 'total import teus')

# Menampilkan TOP 13 destination/export berdasarkan 'total export teus'
top_13_export = pivot_df.nlargest(13, 'total export teus')

# Menghitung akumulasi nilai di luar TOP 13 untuk origin/import
other_import = pd.DataFrame(
    pivot_df[~pivot_df.index.isin(top_13_import.index)].sum().rename('Other')
).T

# Menghitung akumulasi nilai di luar TOP 13 untuk destination/export
other_export = pd.DataFrame(
    pivot_df[~pivot_df.index.isin(top_13_export.index)].sum().rename('Other')
).T

# Menyusun hasil akhir untuk origin/import dengan menambahkan baris 'Other' sebagai baris terakhir
result_import = pd.concat([top_13_import, other_import])
result_import.index.name = 'COUNTRY NAME'

# Menyusun hasil akhir untuk destination/export dengan menambahkan baris 'Other' sebagai baris terakhir
result_export = pd.concat([top_13_export, other_export])
result_export.index.name = 'COUNTRY NAME'

# Kolom yang ingin ditampilkan untuk TOP 13 origin/import
selected_columns_import = [
    'import full teus', 'import empty teus', 'total import teus']

# Kolom yang ingin ditampilkan untuk TOP 13 destination/export
selected_columns_export = [
    'export full teus', 'export empty teus', 'total export teus']

# Menampilkan hasil TOP 13 origin/import dengan baris 'Other' di posisi ke-14
#display(result_import)

# Menampilkan hasil TOP 13 destination/export dengan baris 'Other' di posisi ke-14
#display(result_export)

# Menampilkan TOP 13 origin/import dengan kolom yang dipilih
#print("TOP 13 origin/import")
#result_import_selected = result_import[selected_columns_import]
#result_import_selected.index.name = 'COUNTRY NAME'  # Pastikan kolom COUNTRY NAME terlihat
#display(result_import_selected)

# Menampilkan TOP 13 destination/export dengan kolom yang dipilih
#print("TOP 13 destination/export")
#result_export_selected = result_export[selected_columns_export]
#result_export_selected.index.name = 'COUNTRY NAME'  # Pastikan kolom COUNTRY NAME terlihat
#display(result_export_selected)


# Menyusun TOP 14 untuk memastikan ada kandidat pengganti setelah Indonesia dihapus
top_14_import = pivot_df.nlargest(14, 'total import teus')
top_14_export = pivot_df.nlargest(14, 'total export teus')

# Menghapus negara Indonesia dari TOP 14
top_14_import = top_14_import[top_14_import.index != 'INDONESIA']
top_14_export = top_14_export[top_14_export.index != 'INDONESIA']

# Mengambil 13 negara teratas setelah mengeluarkan Indonesia
top_13_import_without_indonesia = top_14_import.head(13)
top_13_export_without_indonesia = top_14_export.head(13)

# Menambahkan baris 'Other' untuk akumulasi negara lainnya
other_import = pd.DataFrame(
    pivot_df[~pivot_df.index.isin(top_13_import_without_indonesia.index)].sum().rename('Other')
).T
other_export = pd.DataFrame(
    pivot_df[~pivot_df.index.isin(top_13_export_without_indonesia.index)].sum().rename('Other')
).T

# Menggabungkan TOP 13 dengan baris 'Other'
result_import_final = pd.concat([top_13_import_without_indonesia, other_import])
result_import_final.index.name = 'COUNTRY NAME'
result_export_final = pd.concat([top_13_export_without_indonesia, other_export])
result_export_final.index.name = 'COUNTRY NAME'

# Menampilkan hasil akhir
print("import")
display(result_import_final[selected_columns_import])
print("export")
display(result_export_final[selected_columns_export])

# Menyimpan hasil  ke file CSV dan Excel
#result_export_final[selected_columns_export].to_excel('hasil top destination.xlsx')
#result_import_final[selected_columns_import].to_excel('hasil top origin.xlsx')






import


,import full teus,import empty teus,total import teus
COUNTRY NAME,,,
CHINA,54175,45,54220
SINGAPORE,16826,122,16948
"KOREA, REPUBLIC OF",7322,0,7322
MALAYSIA,6956,238,7194
JAPAN,6491,569,7060
INDIA,2241,5,2246
HONG KONG,1810,0,1810
"TAIWAN, PROVINCE OF CHINA",1639,0,1639
UNITED STATES,1085,0,1085


export


,export full teus,export empty teus,total export teus
COUNTRY NAME,,,
CHINA,12049,13858,25907
UNITED STATES,9238,0,9238
PHILIPPINES,5806,1166,6972
"KOREA, REPUBLIC OF",2788,3390,6178
JAPAN,5489,3,5492
INDIA,3501,202,3703
MALAYSIA,3217,248,3465
SINGAPORE,2446,893,3339
VIET NAM,2751,344,3095
